In [1]:
from datasets import load_dataset
from datasets import load_from_disk
import os

f:\stockpredproj\StockPrediction\.stock\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the dataset from the Hugging Face repository <------ uncomment this line to load the dataset

# ds = load_dataset("edaschau/bitcoin_news")
# ds.save_to_disk(os.path.join('train_news_dataset', 'hg_news'))

Saving the dataset (3/3 shards): 100%|██████████| 210832/210832 [00:02<00:00, 90893.93 examples/s] 


In [3]:
# Download dataset from https://github.com/soheilrahsaz/cryptoNewsDataset/tree/main/csvOutput (news_currencies_source_joinedResult.rar) to train_news_dataset dir

In [4]:
import pandas as pd
hg_data = None
github_data = None

In [5]:
github_data = pd.read_csv(os.path.join('train_news_dataset', 'github_news', 'github_news.csv')) 

filtered_github_data = github_data[github_data['currencies'].str.contains('BTC', na=False)]
filtered_github_data = filtered_github_data[['newsDatetime', 'title']]
filtered_github_data = filtered_github_data.rename(columns={'newsDatetime': 'date_time'})

In [6]:
print(os.listdir('train_news_dataset'))

['github_news', 'hg_news', 'merged_news.csv']


In [7]:
hg_data = load_from_disk(os.path.join('train_news_dataset', 'hg_news'))

hg_data = pd.DataFrame(hg_data['train'])
hg_data['date_time'] = hg_data['date_time'].apply(lambda x: x[:-6] if isinstance(x, str) else x)
hg_data = hg_data[['date_time', 'title', 'article_text']]

In [8]:
news = pd.concat([filtered_github_data, hg_data], axis=0)

# Convert 'date_time' to datetime format and filter out rows with invalid date formats
news['date_length'] = news['date_time'].apply(lambda x: len(x)) 
filtered_news = news[news['date_length'] == 19] # filtering for date_time strings of length 19 (to convert to datetime, because 3 of them are not the same format)
filtered_news = filtered_news.drop(columns=['date_length'])
filtered_news['date_time'] = pd.to_datetime(filtered_news['date_time'])

# Sort the DataFrame by 'date_time'
filtered_news = filtered_news.sort_values('date_time')


filtered_news.to_csv(os.path.join('train_news_dataset', 'merged_news.csv'), index=False)
